# Set up Convergence
Make a certain surface, place a adsorbate and then run a force computation.

In [1]:
from examol.utils.conversions import write_to_string
from ase.build import surface, molecule, add_adsorbate
from ase.calculators.cp2k import CP2K
from ase.db import connect
from ase import units

Configuration

In [2]:
xc: str = 'PBE'
basis_set: str = 'DZVP-MOLOPT-SR-GTH'
cutoff: float = 900  # In Ry
rel_cutoff: float = 50

## Make the surface geometry
Have a simple, 4-layered Pt (111) slab with a fair amount of vacuum layers and a methane docked up top. For now, let's explore a fully-periodic.

In [3]:
s1 = surface('Pt', (1, 1, 1), 4)
s1.center(vacuum=12, axis=2)
s1 *= [2, 2, 1]
s1.pbc = True

In [4]:
mol = molecule('CH4')

In [5]:
add_adsorbate(s1, mol, height=4)

## Make the Calculator
CP2K with fully periodic boundary conditions and smearing to account for metal ions.

In [6]:
cp2k_opts = dict(
    xc=None,
    inp=f"""
&FORCE_EVAL
&DFT
  &SCF
    ADDED_MOS -1  ! Add as many as possible
    &SMEAR  ON 
      METHOD FERMI_DIRAC 
      ELECTRONIC_TEMPERATURE [K] 300
    &END SMEAR 
    &MIXING
      METHOD BROYDEN_MIXING
    &END MIXING
  &END SCF
  &XC
     &XC_FUNCTIONAL {xc}
     &END XC_FUNCTIONAL
  &END XC
  &MGRID
    NGRIDS 5
    REL_CUTOFF {rel_cutoff}
  &END MGRID
  &POISSON
     PERIODIC XYZ
     PSOLVER PERIODIC
  &END POISSON
&END DFT
&END FORCE_EVAL""",
    basis_set_file='BASIS_MOLOPT',
    basis_set=basis_set,
    pseudo_potential=f'GTH-{xc}',
    poisson_solver=None,
)
calc = CP2K(directory='conv',
            command='/home/lward/Software/cp2k-2023.2/exe/local_cuda/cp2k_shell.ssmp',
            stress_tensor=True,
            max_scf=250,
            cutoff=cutoff * units.Ry,  # Converts to eV
            **cp2k_opts)

Run to convergence

In [7]:
s1.calc = calc

In [8]:
forces = s1.get_forces()

Save the forces and such

In [9]:
with connect('convergence.db') as db:
    db.write(s1, xc=xc, cutoff=cutoff, rel_cutoff=rel_cutoff, basis_set=basis_set)